# Método dos Elementos Finitos - Trabalho 3

Universidade Federal Fluminense

Disciplina ministrada pelo Prof. Marco Ferro

<marcoferro@id.uff.br>

Aluno Noé de Lima

<noe_lima@id.uff.br>

Este trabalho visa aplicar o MEF a uma estrutura de vigas.

Primeiro semestre de 2020

\vfill

A célula a seguir configura o Jupyter-Notebook para exibir as equações matemáticas no formato do ambiente $\LaTeX$ e importa as bibliotecas necessárias.

In [174]:
import numpy as np
import sympy as sym
import json
sym.init_printing(use_latex='mathjax',latex_mode='equation*')
!uname -a

Linux DESKTOP-CR7O8A2 4.19.104-microsoft-standard #1 SMP Wed Feb 19 06:37:35 UTC 2020 x86_64 x86_64 x86_64 GNU/Linux


\cleardoublepage

\tableofcontents

\cleardoublepage

# Introdução

Em uma estrutura de treliças, cada barra é um elemento, e o Método dos Elementos Finitos coincide com a Análise Matricial de Estruturas. Considerando que todos os esforços na treliça são normais, não há cargas transversais, o problema, visto do ponto de vista das barras, é unidimensional, pois, da mesma forma que os esforços, os deslocamentos também são na mesma direção das barras.

No caso de vigas, havendo momento fletor (ou torçor), tanto direto como em consequência de esforço cortante, há deformação no sentido perpendicular à viga, bem como rotação pontual em decorrência da deformação. Assim, uma viga será discretizada na forma de uma malha compostas por pequenos segmentos.

Em um Elemento de Viga, cujo eixo principal é o eixo $x$, temos:

* $V_{1} \rightarrow$ Força perpendicular no início do Elemento;
* $V_{2} \rightarrow$ Força perpendicular no final do Elemento;
* $P \rightarrow$ Carga perpendicular distribuída;
* $M_{1} \rightarrow$ Momento no início do Elemento;
* $M_{2} \rightarrow$ Momento no final do Elemento;
* $L \rightarrow$ Comprimento do Elemento.

Bem como os deslocamentos:
* $u_{1}=\delta_{1}\rightarrow$ deflexão $\delta$ no início do Elemento;
* $u_{2}=\theta_{1}\rightarrow$ inclinação $\theta$ no início do Elemento;
* $u_{3}=\delta_{2}\rightarrow$ deflexão $\delta$ no final do Elemento;
* $u_{4}=\theta_{2}\rightarrow$ inclinação $\theta$ no final do Elemento.

No caso de uma viga retangular de largura $b$ e altura $h$, temos o momento de inércia:

\begin{equation*}
    I = \frac{bh^{3}}{12}
\end{equation*}

Assim, sendo o Módulo de Elasticidade $E$ constante, o produto $EI$ também será.

Temos, portanto, a seguinte equação diferencial da viga:

\begin{equation*}
    EI\frac{d^{4}u}{dx^{4}} = P
\end{equation*}

Tem-se que:

\begin{equation*}
    V = EI\frac{d^{3}u}{dx^{3}}
\end{equation*}

E

\begin{equation*}
    M = EI\frac{d^{2}u}{dx^{2}}
\end{equation*}

Aplicando a SRP - Sentença de Resíduos Ponderados - na EDO da viga, resulta:

\begin{equation*}
    \int_{\Omega}{N_{i}R}d\Omega = 0
\end{equation*}

$$\therefore$$

\begin{equation*}
    \int_{0}^{L}{N_{i}\left(EI\frac{d^{4}u}{dx^{4}}-P\right)}dx = 0
\end{equation*}


In [175]:
x = sym.symbols('x')
P = sym.symbols('P')
E,I,L = sym.symbols('E,I,L')
u = sym.Function('u')
N_i = sym.symbols('N_i',cls=sym.Function)
R = E*I*sym.diff(u(x),x,4)-P
SRP = sym.integrate(N_i(x)*R,(x,0,L))
display(SRP.doit())

L                             
⌠                             
⎮ ⎛      4          ⎞         
⎮ ⎜     d           ⎟         
⎮ ⎜E⋅I⋅───(u(x)) - P⎟⋅Nᵢ(x) dx
⎮ ⎜      4          ⎟         
⎮ ⎝    dx           ⎠         
⌡                             
0                             

Essa integral não pode ser resolvida pelos métodos sym.dsolve(sym.Eq(SRP,0),u(x)) ou SRP.doit(), pois o nível de complexidade ainda não foi implementado na biblioteca.

Utilizando a integração por partes, obtém-se o seguinte:

In [176]:
eq = sym.integrate(E*I*(sym.diff(N_i(x),x)*sym.diff(u(x),x,3)),(x,0,L))
display(eq.doit())

    L                       
    ⌠                       
    ⎮             3         
    ⎮ d          d          
E⋅I⋅⎮ ──(Nᵢ(x))⋅───(u(x)) dx
    ⎮ dx          3         
    ⎮           dx          
    ⌡                       
    0                       

In [177]:
N_1,N_2,N_3,N_4 = sym.symbols('N_1,N_2,N_3,N_4',cls=sym.Function)
#display(sym.dsolve(sym.Eq(sym.diff(N_1(x),x,2),0),ics={N_1(0):1,N_1(L):0}))
#display(sym.dsolve(sym.Eq(sym.diff(N_2(x),x,2),0),ics={N_2(x).diff(x).subs(x, L):1,N_2(x).diff(x).subs(x, L):0}))
#display(sym.dsolve(sym.Eq(sym.diff(N_3(x),x,2),0),ics={N_3(0):0,N_3(L):1}))
#display(sym.dsolve(sym.Eq(sym.diff(N_4(x),x,2),0),ics={N_4(x).diff(x).subs(x, L):1,N_4(x).diff(x).subs(x, L):0}))

# Matriz de Rigidez Local

In [178]:
K_e = sym.Matrix([[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]])

K_e[0,0] = 12*E*I/L**3
K_e[0,1] = 6*E*I/L**2
K_e[0,2] = -12*E*I/L**3
K_e[0,3] = 6*E*I/L**2

K_e[1,0] = K_e[0,1]
K_e[1,1] = 4*E*I/L
K_e[1,2] = -6*E*I/L**2
K_e[1,3] = 2*E*I/L

K_e[2,0] = K_e[0,2]
K_e[2,1] = K_e[1,2]
K_e[2,2] = 12*E*I/L**3
K_e[2,3] = -6*E*I/L**2

K_e[3,0] = K_e[0,3]
K_e[3,1] = K_e[1,3]
K_e[3,2] = K_e[2,3]
K_e[3,3] = 4*E*I/L

display(K_e/(E*I/L))

⎡ 12    6   -12    6 ⎤
⎢ ──    ─   ────   ─ ⎥
⎢  2    L     2    L ⎥
⎢ L          L       ⎥
⎢                    ⎥
⎢ 6         -6       ⎥
⎢ ─     4   ───    2 ⎥
⎢ L          L       ⎥
⎢                    ⎥
⎢-12   -6    12   -6 ⎥
⎢────  ───   ──   ───⎥
⎢  2    L     2    L ⎥
⎢ L          L       ⎥
⎢                    ⎥
⎢ 6         -6       ⎥
⎢ ─     2   ───    4 ⎥
⎣ L          L       ⎦

# Vetor Local de Forças Nodais

In [179]:
V_1,M_1,V_2,M_2 = sym.symbols('V_1,M_1,V_2,M_2')
f_e = sym.Matrix([V_1,-M_1,-V_2,M_2])

f_e[0] += P*L/2
f_e[1] += P*L**2/12
f_e[2] += P*L/2
f_e[3] += -P*L**2/12

display(f_e)

⎡ L⋅P       ⎤
⎢ ─── + V₁  ⎥
⎢  2        ⎥
⎢           ⎥
⎢  2        ⎥
⎢ L ⋅P      ⎥
⎢ ──── - M₁ ⎥
⎢  12       ⎥
⎢           ⎥
⎢ L⋅P       ⎥
⎢ ─── - V₂  ⎥
⎢  2        ⎥
⎢           ⎥
⎢   2       ⎥
⎢  L ⋅P     ⎥
⎢- ──── + M₂⎥
⎣   12      ⎦

# Exemplo Analítico

A partir da definição da Matriz de Rigidez e do Vetor de Cargas Nodais dentro de um único Elemento $e$, podemos expandir para uma viga (ou diversas vigas) em uma estrutura dividida em $n$ Elementos.

Vamos utilizar, como exemplo, uma viga biapoiada de $2~m$ de comprimento, sendo o apoio no início do vão, de segundo gênero, e o do final do vão, de primeiro gênero. A viga está sujeita a uma carga uniformemente distribuída $P = 10~kN/m$ e tem o produto constante $EI = 10^{2}kNm^{2}$. O problema consiste em calcular o deslocamento e o momento no centro do vão, bem como as reações e as rotações nos apoios.

A solução analítica do problema é:

$$u =$$

In [180]:
u = (5/384)*(P*L**4)/(E*I)
display(u)
display(u.subs(P,10).subs(L,2).subs(E*I,10**2))

                    4  
0.0130208333333333⋅L ⋅P
───────────────────────
          E⋅I          

0.0208333333333333

$$M =$$

In [181]:
M = (P*L**2)/8
display(M)
display(M.subs(P,10).subs(L,2))

 2  
L ⋅P
────
 8  

5

$$\theta_{A} = -\theta_{B} =$$

In [182]:
theta_A = (P*L**3)/(24*E*I)
display(theta_A)
display(theta_A.subs(P,10).subs(L,2).subs(E*I,10**2))

  3   
 L ⋅P 
──────
24⋅E⋅I

1/30

$$V_{A} = V_{B} =$$

In [183]:
V_A = P*L/2
display(V_A)
display(V_A.subs(P,10).subs(L,2))

L⋅P
───
 2 

10

Considerando dois Elementos na solução, temos então:

## Elemento 1

### Variáveis de Deformação

* $u_{1}^{1}\rightarrow$ deflexão $\delta$ no primeiro apoio;

* $u_{2}^{1}\rightarrow$ rotação $\theta$ no primeiro apoio;

* $u_{3}^{1}\rightarrow$ deflexão $\delta$ no centro do vão;

* $u_{4}^{1}\rightarrow$ rotação $\theta$ no centro do vão.

### Variáveis de Carregamento

* $M_{1}^{1}\rightarrow$ Momento Fletor no primeiro apoio;

* $V_{1}^{1}\rightarrow$ Força Cortante no primeiro apoio;

* $M_{2}^{1}\rightarrow$ Momento Fletor no centro do vão;

* $V_{2}^{1}\rightarrow$ Força Cortante no centro do vão.

## Elemento 2

### Variáveis de Deformação

* $u_{1}^{2}\rightarrow$ deflexão $\delta$ no centro do vão;

* $u_{2}^{2}\rightarrow$ rotação $\theta$ no centro do vão;

* $u_{3}^{2}\rightarrow$ deflexão $\delta$ no segundo apoio;

* $u_{4}^{2}\rightarrow$ rotação $\theta$ no segundo apoio.

### Variáveis de Carregamento

* $M_{1}^{2}\rightarrow$ Momento Fletor no centro do vão;

* $V_{1}^{2}\rightarrow$ Força Cortante no centro do vão;

* $M_{2}^{2}\rightarrow$ Momento Fletor no segundo apoio;

* $V_{2}^{2}\rightarrow$ Força Cortante no csegundo apoio.

### Nó Central

 No nó central há a conexão entre o segundo nó do primeiro Elemento e o primeiro nó do segundo Elemento. Portanto, seus valores coincidem. Assim,

\begin{equation*}
    M_{2}^{1} = M_{1}^{2}
\end{equation*}

E

\begin{equation*}
    V_{1}^{2} = V_{2}^{1}
\end{equation*}

## Cálculo das Matrizes de Rigidez Locais

$$\mathbf{K}_{e} = \frac{EI}{L}\times$$

In [184]:
display(K_e/(E*I/L))

⎡ 12    6   -12    6 ⎤
⎢ ──    ─   ────   ─ ⎥
⎢  2    L     2    L ⎥
⎢ L          L       ⎥
⎢                    ⎥
⎢ 6         -6       ⎥
⎢ ─     4   ───    2 ⎥
⎢ L          L       ⎥
⎢                    ⎥
⎢-12   -6    12   -6 ⎥
⎢────  ───   ──   ───⎥
⎢  2    L     2    L ⎥
⎢ L          L       ⎥
⎢                    ⎥
⎢ 6         -6       ⎥
⎢ ─     2   ───    4 ⎥
⎣ L          L       ⎦

## Elemento 1

$$K_{1} = 10^{2}\times$$

In [185]:
K_1 = K_e.subs(E*I,10**2).subs(L,1)
display(K_1/100)

⎡12   6   -12  6 ⎤
⎢                ⎥
⎢ 6   4   -6   2 ⎥
⎢                ⎥
⎢-12  -6  12   -6⎥
⎢                ⎥
⎣ 6   2   -6   4 ⎦

## Elemento 2

$$K_{2} = 10^{2}\times$$

In [186]:
K_2 = K_e.subs(E*I,10**2).subs(L,1)
display(K_1/100)

⎡12   6   -12  6 ⎤
⎢                ⎥
⎢ 6   4   -6   2 ⎥
⎢                ⎥
⎢-12  -6  12   -6⎥
⎢                ⎥
⎣ 6   2   -6   4 ⎦

## Cálculo do Vetor $\vec{f}$

In [187]:
display(f_e)

⎡ L⋅P       ⎤
⎢ ─── + V₁  ⎥
⎢  2        ⎥
⎢           ⎥
⎢  2        ⎥
⎢ L ⋅P      ⎥
⎢ ──── - M₁ ⎥
⎢  12       ⎥
⎢           ⎥
⎢ L⋅P       ⎥
⎢ ─── - V₂  ⎥
⎢  2        ⎥
⎢           ⎥
⎢   2       ⎥
⎢  L ⋅P     ⎥
⎢- ──── + M₂⎥
⎣   12      ⎦

### Elemento 1

In [188]:
V_1_1,M_1_1,V_2_1,M_2_1 = sym.symbols('V_1_1,M_1_1,V_2_1,M_2_1')
f_1 = f_e.subs(L,1).subs(P,10).subs(V_1,V_1_1).subs(V_2,V_2_1).subs(M_1,M_1_1).subs(M_2,M_2_1)
display(f_1)

⎡ V₁ ₁ + 5 ⎤
⎢          ⎥
⎢5/6 - M₁ ₁⎥
⎢          ⎥
⎢ 5 - V₂ ₁ ⎥
⎢          ⎥
⎣M₂ ₁ - 5/6⎦

### Elemento 2

In [189]:
V_1_2,M_1_2,V_2_2,M_2_2 = sym.symbols('V_1_2,M_1_2,V_2_2,M_2_2')
f_2 = f_e.subs(L,1).subs(P,10).subs(V_1,V_1_2).subs(V_2,V_2_2).subs(M_1,M_1_2).subs(M_2,M_2_2)
display(f_2)

⎡ V₁ ₂ + 5 ⎤
⎢          ⎥
⎢5/6 - M₁ ₂⎥
⎢          ⎥
⎢ 5 - V₂ ₂ ⎥
⎢          ⎥
⎣M₂ ₂ - 5/6⎦

### Vetor de Forças Global

In [190]:
f = sym.zeros(6,1)
f[0:4,0] += f_1[0:4,0]
f[2:6,0] += f_2[0:4,0]
display(f)

⎡    V₁ ₁ + 5    ⎤
⎢                ⎥
⎢   5/6 - M₁ ₁   ⎥
⎢                ⎥
⎢V₁ ₂ - V₂ ₁ + 10⎥
⎢                ⎥
⎢  -M₁ ₂ + M₂ ₁  ⎥
⎢                ⎥
⎢    5 - V₂ ₂    ⎥
⎢                ⎥
⎣   M₂ ₂ - 5/6   ⎦

Entretanto, lambramos que no nó central, temos:

\begin{equation*}
    \left[\begin{matrix}
        -V_{2}^{1} & + & V_{1}^{2} & = & 0 \\
        M_{2}^{1} & - & M_{1}^{2} & = & 0
    \end{matrix}\right.
\end{equation*}

Portanto, $\vec{f}$ torna-se:

In [191]:
f_G = f.subs(-V_2_1+V_1_2,0).subs(M_2_1-M_1_2,0)
display(f_G)

⎡ V₁ ₁ + 5 ⎤
⎢          ⎥
⎢5/6 - M₁ ₁⎥
⎢          ⎥
⎢    10    ⎥
⎢          ⎥
⎢    0     ⎥
⎢          ⎥
⎢ 5 - V₂ ₂ ⎥
⎢          ⎥
⎣M₂ ₂ - 5/6⎦

## Matriz de Rigidez Global

A Matriz de Rigidez Global fica:

$$K_G = 10^{2}\times$$

In [192]:
K_G = sym.zeros(6,6)
K_G[0:4,0:4] += K_1
K_G[2:6,2:6] += K_2
display(K_G)

⎡1200   600   -1200  600     0     0  ⎤
⎢                                     ⎥
⎢ 600   400   -600   200     0     0  ⎥
⎢                                     ⎥
⎢-1200  -600  2400    0    -1200  600 ⎥
⎢                                     ⎥
⎢ 600   200     0    800   -600   200 ⎥
⎢                                     ⎥
⎢  0     0    -1200  -600  1200   -600⎥
⎢                                     ⎥
⎣  0     0     600   200   -600   400 ⎦

## Vetor de Deslocamentos

Considerando as condições de contorno, temos:

\begin{equation*}
    \left\{\begin{matrix}
        u_{1}^{1} = u_1 & = & 0 \\
        u_{1}^{2} = u_5 & = & 0 \\
        M_{1}^{1} & = & 0 \\
        M_{2}^{2} & = & 0 \\
    \end{matrix}\right.
\end{equation*}

Portanto,

In [193]:
u_1,u_2,u_3,u_4,u_5,u_6 = sym.symbols('u_1,u_2,u_3,u_4,u_5,u_6')
u = sym.Matrix([u_1,u_2,u_3,u_4,u_5,u_6]).subs(u_1,0).subs(u_5,0)
f_G = f_G.subs(M_1_1,0).subs(M_2_2,0)
display(u)
display(f_G)

⎡0 ⎤
⎢  ⎥
⎢u₂⎥
⎢  ⎥
⎢u₃⎥
⎢  ⎥
⎢u₄⎥
⎢  ⎥
⎢0 ⎥
⎢  ⎥
⎣u₆⎦

⎡V₁ ₁ + 5⎤
⎢        ⎥
⎢  5/6   ⎥
⎢        ⎥
⎢   10   ⎥
⎢        ⎥
⎢   0    ⎥
⎢        ⎥
⎢5 - V₂ ₂⎥
⎢        ⎥
⎣  -5/6  ⎦

Assim, o sistema $\mathbf{K}\cdot\vec{u}=\vec{f}$ torna-se:

In [194]:
eq = sym.Eq(K_G * u, f_G)
display(eq)

⎡600⋅u₂ - 1200⋅u₃ + 600⋅u₄ ⎤   ⎡V₁ ₁ + 5⎤
⎢                          ⎥   ⎢        ⎥
⎢ 400⋅u₂ - 600⋅u₃ + 200⋅u₄ ⎥   ⎢  5/6   ⎥
⎢                          ⎥   ⎢        ⎥
⎢-600⋅u₂ + 2400⋅u₃ + 600⋅u₆⎥   ⎢   10   ⎥
⎢                          ⎥ = ⎢        ⎥
⎢ 200⋅u₂ + 800⋅u₄ + 200⋅u₆ ⎥   ⎢   0    ⎥
⎢                          ⎥   ⎢        ⎥
⎢-1200⋅u₃ - 600⋅u₄ - 600⋅u₆⎥   ⎢5 - V₂ ₂⎥
⎢                          ⎥   ⎢        ⎥
⎣ 600⋅u₃ + 200⋅u₄ + 400⋅u₆ ⎦   ⎣  -5/6  ⎦

A partir deste sistema, considerando as condições de contorno, podemos obter o seguinte sistema reduzido:

In [195]:
f_R = f_G.copy()
f_R -= u[0,0]*K_G[:,0]
f_R -= u[4,0]*K_G[:,4]

K_R = K_G.copy()
K_R.row_del(4)
K_R.col_del(4)
K_R.row_del(0)
K_R.col_del(0)
f_R.row_del(4)
f_R.row_del(0)

display(K_R)
display(f_R)

⎡400   -600  200   0 ⎤
⎢                    ⎥
⎢-600  2400   0   600⎥
⎢                    ⎥
⎢200    0    800  200⎥
⎢                    ⎥
⎣ 0    600   200  400⎦

⎡5/6 ⎤
⎢    ⎥
⎢ 10 ⎥
⎢    ⎥
⎢ 0  ⎥
⎢    ⎥
⎣-5/6⎦

A solução deste sistema fornece:

$$u' = 10^{-2}\times$$

In [196]:
u_R = K_R.solve(f_R)
display(u_R*100)

⎡10/3 ⎤
⎢     ⎥
⎢ 25  ⎥
⎢ ──  ⎥
⎢ 12  ⎥
⎢     ⎥
⎢  0  ⎥
⎢     ⎥
⎣-10/3⎦

Temos, portanto,

\begin{equation*}
    \left\{\begin{matrix}
        u_{2}^{1} = u_{2} & = & \frac{10}{3}\times10^{-2} rad \\
        u_{3}^{1} = u_{1}^{2} = u_{3} & = & \frac{25}{12}\times10^{-2} m \\
        u_{4}^{1} = u_{2}^{2} = u_{4} & = & 0 \\
        u_{4}^{2} = u_{6} & = & -\frac{10}{3}\times10^{-2} rad \\
    \end{matrix}\right.
\end{equation*}

Temos, agora, o seguinte vetor de deslocamentos:

$$u = 10^{-2}$$

In [197]:
u = u.subs(u_2,1/30).subs(u_3,25/1200).subs(u_4,0).subs(u_6,-1/30)
display(100*u)

⎡        0        ⎤
⎢                 ⎥
⎢3.33333333333333 ⎥
⎢                 ⎥
⎢2.08333333333333 ⎥
⎢                 ⎥
⎢        0        ⎥
⎢                 ⎥
⎢        0        ⎥
⎢                 ⎥
⎣-3.33333333333333⎦

Agora podemos calcular as forças nodais.

$$\vec{f} = $$

In [198]:
f = K_G*u
display(f,f_G)

⎡       -5.0       ⎤
⎢                  ⎥
⎢0.833333333333334 ⎥
⎢                  ⎥
⎢       10.0       ⎥
⎢                  ⎥
⎢        0         ⎥
⎢                  ⎥
⎢       -5.0       ⎥
⎢                  ⎥
⎣-0.833333333333334⎦

⎡V₁ ₁ + 5⎤
⎢        ⎥
⎢  5/6   ⎥
⎢        ⎥
⎢   10   ⎥
⎢        ⎥
⎢   0    ⎥
⎢        ⎥
⎢5 - V₂ ₂⎥
⎢        ⎥
⎣  -5/6  ⎦

Temos, portanto,

\begin{equation*}
    \left\{\begin{matrix}
        V_{1}^{1} + 5 & = & -5 \\
        5 - V_{2}^{2} & = & -5
    \end{matrix}\right.
\end{equation*}

Logo,

\begin{equation*}
    \left\{\begin{matrix}
        V_{1}^{1} & = & -10 \\
        V_{2}^{2} & = & 10 \\
    \end{matrix}\right.
\end{equation*}

Temos, portanto, os seguintes valores de $\vec{u}_{e}^{1}$ no Elemento 1:

$$u_e^{1} =$$

In [199]:
u_e_1 = u[0:4,0]
display(u_e_1)

⎡        0         ⎤
⎢                  ⎥
⎢0.0333333333333333⎥
⎢                  ⎥
⎢0.0208333333333333⎥
⎢                  ⎥
⎣        0         ⎦

Da mesma forma, temos como calcular o vetor de Forças Nodais $\vec{f}_{e}^{1}$.

$$\vec{f}_{e}^{1} =$$

In [200]:
f_e_1 = K_1*u_e_1
display(f_e_1,f_e.subs(L,1).subs(P,10))

⎡      -5.0       ⎤
⎢                 ⎥
⎢0.833333333333334⎥
⎢                 ⎥
⎢       5.0       ⎥
⎢                 ⎥
⎣-5.83333333333333⎦

⎡ V₁ + 5 ⎤
⎢        ⎥
⎢5/6 - M₁⎥
⎢        ⎥
⎢ 5 - V₂ ⎥
⎢        ⎥
⎣M₂ - 5/6⎦

De onde resulta:

\begin{equation*}
    \left\{\begin{matrix}
        V_{1}^{1} & = & -10~kN \\
        M_{1}^{1} & = & 0 \\
        V_{2}^{1} & = & 0 \\
        M_{2}^{1} & = & -5~kN\cdot m
    \end{matrix}\right.
\end{equation*}

Seguindo o mesmo raciocínio, podemos calcular o vetor de Forças Nodais do Elemento 2, $\vec{f}_{e}^{2}$.

$$\vec{f}_{e}^{2} =$$

In [201]:
u_e_2 = u[2:6,0]
f_e_2 = K_2*u_e_2
display(f_e_2,f_e.subs(L,1).subs(P,10))

⎡       5.0        ⎤
⎢                  ⎥
⎢ 5.83333333333333 ⎥
⎢                  ⎥
⎢       -5.0       ⎥
⎢                  ⎥
⎣-0.833333333333334⎦

⎡ V₁ + 5 ⎤
⎢        ⎥
⎢5/6 - M₁⎥
⎢        ⎥
⎢ 5 - V₂ ⎥
⎢        ⎥
⎣M₂ - 5/6⎦

De onde resulta:

\begin{equation*}
    \left\{\begin{matrix}
        V_{1}^{2} & = & 0 \\
        M_{1}^{2} & = & -5~kN\cdot m \\
        V_{2}^{2} & = & 10~kN \\
        M_{2}^{2} & = & 0
    \end{matrix}\right.
\end{equation*}

# Solução Por Código

A seguir, vamos elaborar um código em Python para solucionar uma viga a partir de um arquivo de entrada no formato JSON cotendo as informações da estrutura.

vamos começar definindo uma classe para conter os nós da estrutura.

In [203]:
# Class to store and update nodes and nodes data as locate, if have constrains and loads, as well calculated reactions and deslocs
class node:
    def __init__(self,i,P=np.array([0,0,0]),S=[False,False,False,False,False,False],L=np.array([0,0,0,0,0,0]),tag=''):
            self.index = i                          # node index reference
            self.locate = P                         # (x,y,z)
            self.support = S                        # (Rx,Ry,Rz,Rmx,Rmy,Rmz) bool if exists
            self.load = L                           # (Fx,Fy,Fz,Mx,My,Mz)
            self.tag = tag                          # String name
            self.reaction = np.array([0,0,0,0,0,0]) # (Rx,Ry,Rz,Rmx,Rmy,Rmz) to calculate
            self.u = np.array([0,0,0,0,0,0])        # (dx,dy,dz,theta_x,theta_y,theta_z) to calculate
    
    def Reaction(self,R=np.array([0,0,0,0,0,0])):
        self.reaction = R                           # Calculated values

    def Desloc(self,L=np.array([0,0,0,0,0,0])):
        self.load = L                               # Calculated values

Da mesma forma, precisamos de uma classe para operar com as barras que conectam os nós, assim como as propriedades de carga e deformação.

A classe abaixo lida com barras como elementos finitos locais. As barras reais serão criadas como um elemento único e subdivididas em elementos com nós intermediários.

In [ ]:
# Class to store and calculate bars or elements
class element:
    def __init__(self,i,first,last,P=np.array([0,0,0]),tag=''):
        self.index = i
        self.start = first
        self.end = last
        self.load = P

    # To K matrix, we are considerating the variables below:
    # * u_0 -> local dx_1
    # * u_1 -> local dy_1
    # * u_2 -> local dz_1
    # * u_3 -> local theta_x_1 (yz)
    # * u_4 -> local theta_y_1 (zx)
    # * u_5 -> local theta_z_1 (xy)
    # * u_6 -> local dx_2
    # * u_7 -> local dy_2
    # * u_8 -> local dz_2
    # * u_9 -> local theta_x_2 (yz)
    # * u_10 -> local theta_y_2 (zx)
    # * u_11 -> local theta_z_2 (xy)
    # two-dimensional plan just use u_0, u_1, u_3, u_6, u_7, u_11
    def K(self):
        K = np.array([
            [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]
            [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]
            [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]
            [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]
            [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]
            [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]
            [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]
            [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]
            [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]
            [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]
            [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]
            [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]
        ])
        for i in range(12): # Symetrical matrix loop
            for j in range(i):
                K[i,j] = K[j,i]
        return K